In [24]:
import os

project_root = "/Users/nimisha/Desktop/learning/news-reliability-detector/"
os.chdir(project_root)

In [25]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lower, regexp_replace

# local modules
from src.load.load_data import load
from src.preprocessing.preprocessing import (
    print_null_rows,
    drop_missing_text_rows,
    shuffle_rows,
)

In [26]:
spark = SparkSession.builder.appName("NewsReliability").getOrCreate()

In [27]:
data = load(spark)

In [28]:
data.show(10)

+--------------------+--------------------+-------+-----------------+------+
|               title|                text|subject|             date|target|
+--------------------+--------------------+-------+-----------------+------+
| Donald Trump Sen...|Donald Trump just...|   News|December 31, 2017|     1|
| Drunk Bragging T...|House Intelligenc...|   News|December 31, 2017|     1|
| Sheriff David Cl...|On Friday, it was...|   News|December 30, 2017|     1|
| Trump Is So Obse...|On Christmas day,...|   News|December 29, 2017|     1|
| Pope Francis Jus...|Pope Francis used...|   News|December 25, 2017|     1|
| Racist Alabama C...|The number of cas...|   News|December 25, 2017|     1|
| Fresh Off The Go...|Donald Trump spen...|   News|December 23, 2017|     1|
| Trump Said Some ...|In the wake of ye...|   News|December 23, 2017|     1|
| Former CIA Direc...|Many people have ...|   News|December 22, 2017|     1|
| WATCH: Brand-New...|Just when you mig...|   News|December 21, 2017|     1|

In [29]:
data.printSchema()

root
 |-- title: string (nullable = true)
 |-- text: string (nullable = true)
 |-- subject: string (nullable = true)
 |-- date: string (nullable = true)
 |-- target: integer (nullable = false)



# Handling Missing Values

In [30]:
print_null_rows(data)

+-----+----+-------+----+------+
|title|text|subject|date|target|
+-----+----+-------+----+------+
|    0|   8|      8|   8|     0|
+-----+----+-------+----+------+



In [31]:
data.filter(data.text.isNull()).show()

+--------------------+----+-------+----+------+
|               title|text|subject|date|target|
+--------------------+----+-------+----+------+
|                   "|NULL|   NULL|NULL|     1|
|Ex-GOP Congressma...|NULL|   NULL|NULL|     1|
|                   "|NULL|   NULL|NULL|     1|
|WATCH: Democratic...|NULL|   NULL|NULL|     1|
|                   "|NULL|   NULL|NULL|     1|
|Trump Gets STOMPE...|NULL|   NULL|NULL|     1|
|                   "|NULL|   NULL|NULL|     1|
|Donald Trump Gets...|NULL|   NULL|NULL|     1|
+--------------------+----+-------+----+------+



In [32]:
data = drop_missing_text_rows(data)

In [33]:
data.filter(data.text.isNull()).show()

+-----+----+-------+----+------+
|title|text|subject|date|target|
+-----+----+-------+----+------+
+-----+----+-------+----+------+



In [34]:
print_null_rows(data)

+-----+----+-------+----+------+
|title|text|subject|date|target|
+-----+----+-------+----+------+
|    0|   0|      0|   0|     0|
+-----+----+-------+----+------+



# Randomize the order of data rows

In [35]:
data = shuffle_rows(data)
data.show(10)

+--------------------+--------------------+------------+-------------------+------+
|               title|                text|     subject|               date|target|
+--------------------+--------------------+------------+-------------------+------+
|U.S. tax debate e...|WASHINGTON (Reute...|politicsNews|  December 1, 2017 |     0|
|OOPS! VIDEO Shows...|In spite of Senat...|   left-news|       Jul 28, 2017|     1|
|Clinton's lead ov...|NEW YORK (Reuters...|politicsNews|    August 5, 2016 |     0|
|White House will ...|WASHINGTON (Reute...|politicsNews|      May 16, 2017 |     0|
|Venezuelans face ...|PUERTO ORDAZ, Ven...|   worldnews|September 21, 2017 |     0|
|Clinton says 'con...|(Reuters) - Hilla...|politicsNews| September 5, 2016 |     0|
|U.S. Senate Passe...|21st Century Wire...| Middle-east|     March 18, 2016|     1|
|GAME ON! UC Berke...|THE BIRTHPLACE OF...|    politics|       Apr 19, 2017|     1|
|Mexico urges resp...|(Reuters) - U.S. ...|politicsNews|     April 6, 2017 |

# Drop columns

In [36]:
columns_to_drop = ["date", "title"]
data = data.drop(*columns_to_drop)
data.show(2)

+--------------------+------------+------+
|                text|     subject|target|
+--------------------+------------+------+
|WASHINGTON (Reute...|politicsNews|     0|
|In spite of Senat...|   left-news|     1|
+--------------------+------------+------+
only showing top 2 rows



# Duplicate Rows

In [37]:
duplicate_rows = data.groupBy(data.columns).count().filter("count > 1")

print("Duplicates", duplicate_rows.count())
print("Total", data.count())
print("Distinct", data.distinct().count())

Duplicates 465
Total 44899
Distinct 43782


In [38]:
duplicate_rows.show(10)

+--------------------+--------------------+------+-----+
|                text|             subject|target|count|
+--------------------+--------------------+------+-----+
|"#JokeNewsAfter r...| 2017The former M...|     1|    2|
|"Liberal actor an...| 2017Harvey Weins...|     1|    2|
|"President Trump ...| we're not lookin...|     1|    2|
|"Judge Jeanine le...| and?"" he says. ...|     1|    2|
|"Local artist Ric...| 2017Later on Twi...|     1|    2|
|"Brigitte Macron ...|""  THIS: pic.twi...|     1|    2|
|" I m a political...|               2017"|     1|    2|
|"This unbelievabl...|               2016"|     1|    2|
|"Remember the tim...|     fjs);}(document|     1|    2|
|"Barack Obama has...|        undocumented|     1|    2|
+--------------------+--------------------+------+-----+
only showing top 10 rows



In [39]:
data = data.dropDuplicates()
data.count()

43782

# Text Transform 

### 1. Lower Case

In [40]:
# convert to all lower case
data = data.withColumn("text", lower(col("text")))
data = data.withColumn("subject", lower(col("subject")))
data.show(5)

+--------------------+-------+------+
|                text|subject|target|
+--------------------+-------+------+
|president donald ...|   news|     1|
|the republican fa...|   news|     1|
|there s a lot of ...|   news|     1|
|everyone knows wh...|   news|     1|
|donald trump took...|   news|     1|
+--------------------+-------+------+
only showing top 5 rows



### 2. Remove Special Characters 
- like ! @

In [ ]:
data = data.withColumn("text", regexp_replace("text", "[^a-zA-Z0-9\\s]", ""))
data.show(5)

+--------------------+-------+------+
|                text|subject|target|
+--------------------+-------+------+
|president donald ...|   news|     1|
|the republican fa...|   news|     1|
|there s a lot of ...|   news|     1|
|everyone knows wh...|   news|     1|
|donald trump took...|   news|     1|
+--------------------+-------+------+
only showing top 5 rows

